<a href="https://colab.research.google.com/github/harshil-1402/Food-Delivery-Chatbot/blob/main/Food_Delivery_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk

In [ ]:
pwd

'/'

In [ ]:
cd content/

/content


In [ ]:
cd sample_data/

[Errno 2] No such file or directory: 'sample_data/'
/


In [ ]:
cd ../

/


In [ ]:
import random
import json
import numpy as np
import pickle
from tensorflow.keras.optimizers import legacy
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []

ignore_letters=['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

classes=sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


training=[]
output_empty=[0]*len(classes)

for document in documents:
    bag=[]
    word_patterns=document[0]
    #print("xxxxxxxx", word_patterns)
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    #print("yyyyyyyyyy", word_patterns)
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

random.shuffle(training)
training=np.array(training)

train_x=list(training[:, 0])
train_y=list(training[:, 1])


model=Sequential()

model.add(Dense(128, input_shape= (len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save('chatbot_model.h5', hist)
print('done')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-1-cd9b201baacf>:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
5/5 [==============================] - 1s 6ms/step - loss: 1.5514 - accuracy: 0.3333
Epoch 2/200
5/5 [==============================] - 0s 5ms/step - loss: 1.5316 - accuracy: 0.3333
Epoch 3/200
5/5 [==============================] - 0s 5ms/step - loss: 1.5013 - accuracy: 0.3750
Epoch 4/200
5/5 [==============================] - 0s 5ms/step - loss: 1.4671 - accuracy: 0.5417
Epoch 5/200
5/5 [==============================] - 0s 5ms/step - loss: 1.4769 - accuracy: 0.4583
Epoch 6/200
5/5 [==============================] - 0s 6ms/step - loss: 1.3635 - accuracy: 0.5833
Epoch 7/200
5/5 [==============================] - 0s 4ms/step - loss: 1.4034 - accuracy: 0.5833
Epoch 8/200
5/5 [==============================] - 0s 4ms/step - loss: 1.3117 - accuracy: 0.5833
Epoch 9/200
5/5 [==============================] - 0s 4ms/step - loss: 1.2275 - accuracy: 0.6250
Epoch 10/200
5/5 [==============================] - 0s 5ms/step - loss: 1.1754 - accuracy: 0.6667
Epoch 11/200
5/5 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd

In [ ]:
import random
from tensorflow.keras.models import load_model
lemmatizer=WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words=pickle.load(open('words.pkl', 'rb'))
classes=pickle.load(open('classes.pkl', 'rb'))
model=load_model('chatbot_model.h5')

print(classes)

def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word==w:
        bag[i]=1

  return np.array(bag)

def predict_class(sentence):
  bow= bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i, r]for i, r in enumerate(res)]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list


def get_response(intents_list, intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

print('Bot is running')
while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints, intents)
  print(res)

['goodbye', 'greetings', 'new order', 'track order', 'update order']
Bot is running
Hi
1/1 [==============================] - 0s 97ms/step
Hey!
new order
1/1 [==============================] - 0s 19ms/step
here are your options! pav bhaji, choe bhature
pav bhaji
1/1 [==============================] - 0s 20ms/step
what can I do for you?
track order
1/1 [==============================] - 0s 20ms/step
tell your order id
41
1/1 [==============================] - 0s 21ms/step
what can I do for you?
That's all
1/1 [==============================] - 0s 19ms/step
cya!


KeyboardInterrupt: Interrupted by user

# New Section